# Reference  
* [DMI指标.py](https://github.com/zhy0313/ea-python/blob/master/DMI%E6%8C%87%E6%A0%87.py)  
* [DMI: 一个重要的技术指标](https://baijiahao.baidu.com/s?id=1697807373881020020)  
* [ta-lib-python](https://github.com/TA-Lib/ta-lib-python)  
* [輕鬆學會程式交易 | Chapter 5 | 建立自己的交易系統](https://hackmd.io/@tai-quantup/ch5)  
* [量化交易策略](https://github.com/zhy0313/ea-python)

In [ ]:
import os
from datetime import date
from datetime import datetime

import pandas as pd

In [ ]:
from finlab import data

In [ ]:
# 引用自建公用模組
from proj_util_pkg.settings import ProjEnvSettings
from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.google_api import gspread_manager as gsm
from proj_util_pkg.common import tw_stock_topic as tst

## 公用參數設定

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [ ]:
# 資訊輸出Google SpreadSheet 表單參數設定
GSPERAD_SHEET_KEY = os.environ.get('gspread_wb_key')  # Google SpreadSheet 表單ID
OUTPUT_GSHEET_NAME = '選股清單02'

In [ ]:
# 本地報表輸出路徑
REPORT_PATH = os.environ.get('report_path')
REPORT_PATH

## 外部資料讀取

In [ ]:
# 讀取台股收盤價資訊
close = data.get("price:收盤價", save_to_storage=True)
vol = data.get("price:成交股數", save_to_storage=True)
stock_info = data.get('company_basic_info', save_to_storage=True)
pe_ratio = data.get('price_earning_ratio:本益比', save_to_storage=True)
pb_ratio = data.get('price_earning_ratio:股價淨值比', save_to_storage=True)
institutional_investors_foreign = data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)', save_to_storage=True)
institutional_investors_inv_trust = data.get('institutional_investors_trading_summary:投信買賣超股數', save_to_storage=True)
institutional_investors_dealer = data.get('institutional_investors_trading_summary:自營商買賣超股數(避險)', save_to_storage=True)

## 數據分析

In [ ]:
minus_di = data.indicator(
    "MINUS_DI",
    adjust_price=False,
    resample="D",
    timeperiod=72,
    save_to_storage=True,
)

In [ ]:
plus_di = data.indicator(
    "PLUS_DI",
    adjust_price=False,
    resample="D",
    timeperiod=72,
    save_to_storage=True,
)

In [ ]:
sma10 = close.average(10)
sma20 = close.average(20)
vol_sma5 = vol.average(5)

In [ ]:
# 黃金交叉
def crossover(over,down):
    a1 = over
    b1 = down
    a2 = a1.shift(1)
    b2 = b1.shift(1)
    crossover =  (a1>a2) & (a1>b1) & (b2>a2)

    return crossover

# 死亡交叉
def crossunder(down,over):
    a1 = down
    b1 = over
    a2 = a1.shift(1)
    b2 = b1.shift(1)
    crossdown =  (a1<a2) & (a1<b1) & (b2<a2)
    
    return crossdown

In [ ]:
# crossover_test = crossover(sma10, sma20)
# crossover_test
# filter_stock = crossover_test.tail(1)
# filtered_symbols = crossover_test.columns[crossover_test.iloc[0]].tolist()
# df_filtered_symbols = pd.DataFrame(filtered_symbols, columns=['symbol'])
# df_filtered_symbols

In [ ]:
# 五日均量大於500張
# vol.average(5) > 500000

In [ ]:
cond_1 = plus_di.rise().sustain(2)
cond_2 = plus_di > minus_di
# cond_3 = sma10.rise().sustain(3)
# cond_4 = sma20.rise().sustain(3)
# cond_5 = sma10 > sma20
cond_5 = crossover(sma10, sma20)
# cond_6 = crossover(plus_di, minus_di)

filter_stock = cond_1 & cond_2 & cond_5
filter_stock

In [ ]:
filter_stock = filter_stock.tail(1)
filtered_symbols = filter_stock.columns[filter_stock.iloc[0]].tolist()
df_filtered_symbols = pd.DataFrame(filtered_symbols, columns=['symbol'])
df_filtered_symbols

In [ ]:
stock_name = stock_info[['stock_id', '公司簡稱']]
stock_name = stock_name.rename(columns={'stock_id': 'symbol'})

In [ ]:
last_vol_sma5 = vol_sma5.tail(1).T.reset_index()
last_vol_sma5 = last_vol_sma5.rename(columns={last_vol_sma5.columns[0]: 'symbol', last_vol_sma5.columns[1]: 'vol_sma5'})
last_vol_sma5.fillna(0, inplace=True)
# last_vol_sma5["五日均量"] = last_vol_sma5["vol_sma5"] / 1000
last_vol_sma5['五日均量'] = last_vol_sma5['vol_sma5'] / 1000
last_vol_sma5['五日均量'] = last_vol_sma5['五日均量'].round().astype(int)

del last_vol_sma5['vol_sma5']

last_vol_sma5

In [ ]:
# Assuming you have a DataFrame called stock_name with columns stock_id and stock_name

# Merge df_filtered_symbols with stock_name on stock_id
merged_df = df_filtered_symbols.merge(stock_name, on='symbol')
merged_df = merged_df.merge(last_vol_sma5, on='symbol')
merged_df["web_link"] = merged_df["symbol"].apply(lambda x: f"https://www.wantgoo.com/stock/{x}/technical-chart")
merged_df["題材概念股"] = merged_df["symbol"].apply(lambda x: tst.read_topic_stocks(x))

# Print the merged DataFrame
print(merged_df)

In [ ]:
# 輸出報表留存
today = datetime.now().strftime("%Y%m%d")
merged_df.to_excel(f'{REPORT_PATH}/選股02_{today}.xlsx', index=False)

## 輸出結果至Google sheet

In [ ]:
# Google SpreadSheet 公用程式初始化
gspread_mgr = gsm.GspreadManager()
gspread_wb = gspread_mgr.get_spreadsheet(GSPERAD_SHEET_KEY)

print(f"更新Google 表單：{gspread_wb.title}，工作表：{OUTPUT_GSHEET_NAME}")

In [ ]:
# 刪除再重建工作表
gspread_mgr.recreate_worksheet(GSPERAD_SHEET_KEY, OUTPUT_GSHEET_NAME)

In [ ]:
# 更新工作表資料
gspread_mgr.update_worksheet_values(
    GSPERAD_SHEET_KEY, 
    OUTPUT_GSHEET_NAME, 
    [merged_df.columns.values.tolist()] + merged_df.values.tolist()
)